In [1]:
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance
import pandas as pd
import glob
import pickle
import json

In [2]:
claim_file = "SciFact/claims_test.jsonl"
corpus_file = "SciFact/corpus.jsonl"

In [3]:
corpus = {}
with open(corpus_file) as f:
    for line in f:
        abstract = json.loads(line)
        corpus[str(abstract["doc_id"])] = abstract

In [4]:
claims = []
with open(claim_file) as f:
    for line in f:
        claim = json.loads(line)
        claims.append(claim)

In [5]:
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

In [6]:
model_path = "/nas/evidx/corpora/XiangciLi/sentence_embedding/BioSentVec_PubMed_MIMICIII-bigram_d700.bin"
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

model successfully loaded


In [7]:
stop_words = set(stopwords.words('english'))

In [ ]:
# By paragraph embedding
corpus_embeddings = {}
for k, v in corpus.items():
    original_sentences = [v['title']] + v['abstract']
    processed_paragraph = " ".join([preprocess_sentence(sentence) for sentence in original_sentences])
    sentence_vector = model.embed_sentence(processed_paragraph)
    corpus_embeddings[k] = sentence_vector

In [ ]:
with open("corpus_paragraph_biosentvec.pkl","wb") as f:
    pickle.dump(corpus_embeddings,f)

In [8]:
claim_embeddings = {}
for claim in claims:
    processed_sentence = preprocess_sentence(claim['claim'])
    sentence_vector = model.embed_sentence(processed_sentence)
    claim_embeddings[claim["id"]] = sentence_vector

In [9]:
with open("test_claim_biosentvec.pkl","wb") as f:
    pickle.dump(claim_embeddings,f)